In [3]:
#SageWidget-Calculus-VolumeByShells-09-04-2019-0040-LU
from sage.plot.plot3d.shapes import *; var('x', 'y', 'z', 'u', 'v','w', 't', 's'); 
pretty_print(html('<h2><font face="Comic Sans MS">Volume by Revolution: Cylindrical Shells</font></h2>'))
def draw_washer(r, R, ax, h, c, o):
        w = point3d((0,0,0),size=5,color='white') # Show the origin
        if r < R:
            w += parametric_plot3d([u*sin(v), ax + u*cos(v), 0], (u, r, R), (v, 0, 2 * pi), color = c, opacity = o) # BOTTOM
            w += parametric_plot3d([u*sin(v), ax + u*cos(v), h], (u, r, R), (v, 0, 2 * pi), color = c, opacity = o) # TOP
            w += parametric_plot3d([R * sin(v), ax + R * cos(v), h * t], (t, 0, 1), (v, 0, 2 * pi), color = c, opacity = o) # OUTSIDE
            w += parametric_plot3d([r * sin(v), ax + r * cos(v), h * t], (t, 0, 1), (v, 0, 2 * pi), color = c, opacity = o) # INSIDE
        return w
@interact(layout = [['rotation']]) 
def _(rotation = selector(['|','--'], label=r'$Axis\ of\ Rotation$', buttons=True)):
    cc = ['#FB3535', '#FDA819', '#67B497','#FAE7A8', '#3B3028']
    if rotation == '|':
        @interact(layout = [['axisR'], ['y1', 'y2', 'xint', 'space'], ['showVolume','showShells', 'n', 'xstar']])
        def _(axisR = input_box(0, width=3, label=r'$Axis\ of\ rotation: x=$'),
              y1 = input_box(x, width=20, label=r'$y_1=$'), 
              y2 = input_box(sqrt(x), width=20, label=r'$y_2=$'),
              xint = input_box((0, 1), width=10, label=r'$x:$'),
              xstar = selector(['Arbitrary', 'Lower endpoint', 'Upper endpoint', 'Midpoint'], label=r'$x^*$'),
              n = input_box(5, width=3, label=r'$n$'),
              showVolume = (r'$Volume$', True),
              showShells = (r'$Shells$', False)):
            (a, b) = xint
            dx = (b - a) / n
            y1 = y1 + 0 * x # avoid error when the integrand is a constant.
            y2 = y2 + 0 * x # avoid error when the integrand is a constant.
            y1 = y1.function(x)
            y2 = y2.function(x)
            curves = parametric_plot3d([0, x, y1(x)], (x, a, b), radius=0.01, color='red')
            curves += parametric_plot3d([0, x, y2(x)], (x, a, b), radius=0.01, color='blue')
            bb = curves.bounding_box()
            (xmin, xmax, ymin, ymax) = [bb[0][1], bb[1][1], bb[0][2], bb[1][2]]
            p = line3d([(0, axisR, ymin - (ymax - ymin) / 2), (0, axisR, ymax + (ymax - ymin) / 2)], radius=0.001, color='black') # rotation axis
            p += arrow3d((0, 0, min(0, ymin)), (0, 0, 1.1 * max(abs(ymin), abs(ymax))), radius=0.005, color='black') # y-axis
            p += arrow3d((0, min(0, xmin), 0), (0, 1.1 * max(abs(xmin), abs(xmax)), 0), radius=0.005, color='black') # x-axis
            if showVolume == True:
                p += parametric_plot3d([abs(u - axisR) * cos(v), axisR + abs(u - axisR) * sin(v), y1(u)], (u, a, b), (v, 0, 2*pi), color='#ABB2B9', opacity=0.8) # surface 1
                p += parametric_plot3d([abs(u - axisR) * cos(v), axisR + abs(u - axisR) * sin(v), y2(u)], (u, a, b), (v, 0, 2*pi), color='#ABB2B9', opacity=0.8) # surface 2
                if y1(b) != y2(b):
                    p += parametric_plot3d([abs(b - axisR) * cos(v), axisR + abs(b - axisR) * sin(v), min(y1(b), y2(b)) + t * (max(y1(b), y2(b)) - min(y1(b), y2(b)))], (t, 0, 1), (v, 0, 2*pi), color='#ABB2B9', opacity=0.8) # outside
                if y1(a) != y2(a):
                    p += parametric_plot3d([abs(a - axisR) * cos(v), axisR + abs(a - axisR) * sin(v), min(y1(a), y2(a)) + t * (max(y1(a), y2(a)) - min(y1(a), y2(a)))], (t, 0, 1), (v, 0, 2*pi), color='#ABB2B9', opacity=0.8) # inside
            if showShells == True:
                if xstar == 'Lower endpoint':
                    xx = [a + dx * i for i in range(n)]   
                elif xstar == 'Upper endpoint':
                    xx = [a + dx * (i + 1) for i in range(n)] 
                elif xstar == 'Midpoint':
                    xx = [a + dx * (i + 1/2) for i in range(n)]
                else:
                    xx = [a + dx * (i + random()) for i in range(n)]
                p += sum(draw_washer( abs(a - axisR) + i * dx, abs(a - axisR) + (i + 1) * dx, axisR, abs(y1(xx[i]) - y2(xx[i])), cc[mod(i, 5)], 0.8).translate(0, 0, min(y1(xx[i]), y2(xx[i]))) for i in range(n))
                if n <= 5:
                    p += sum(line3d([(0, xx[i], y1(xx[i])),(0, xx[i], y2(xx[i]))], radius=0.005, color='black') for i in range(n))
            show(p + curves, frame = False, viewer = 'threejs', aspect_ratio = 1)
    elif rotation == '--':
        @interact(layout = [['axisR'], ['x1', 'x2', 'yint', 'space'], ['showVolume','showShells', 'n', 'ystar']])
        def _(axisR = input_box(0, width=3, label=r'$Axis\ of\ rotation: y=$'),
              x1 = input_box(y, width=20, label=r'$x_1=$'), 
              x2 = input_box(y^2, width=20, label=r'$x_2=$'),
              yint = input_box((0, 2), width=10, label=r'$y:$'),
              ystar = selector(['Arbitrary', 'Lower endpoint', 'Upper endpoint', 'Midpoint'], label=r'$y^*$'),
              n = input_box(5, width=3, label=r'$n$'),
              showVolume = (r'$Volume$', True),
              showShells = (r'$Shells$', False)):
            (a, b) = yint
            dy = (b - a) / n
            x1 = x1 + 0 * y # avoid error when the integrand is a constant.
            x2 = x2 + 0 * y # avoid error when the integrand is a constant.
            x1 = x1.function(y)
            x2 = x2.function(y)
            curves = parametric_plot3d([0, x1(y), y], (y, a, b), radius=0.01, color='red')
            curves += parametric_plot3d([0, x2(y), y], (y, a, b), radius=0.01, color='blue')
            bb = curves.bounding_box()
            (xmin, xmax, ymin, ymax) = [bb[0][1], bb[1][1], bb[0][2], bb[1][2]]
            p = line3d([(0, xmin - (xmax - xmin) / 2, axisR), (0, xmax + (xmax - xmin) / 2, axisR)], radius=0.001, color='black') # rotation axis
            p += arrow3d((0, min(0, xmin), 0), (0, 1.1 * max(abs(xmin), abs(xmax)), 0), radius=0.005, color='black') # x-axis
            p += arrow3d((0, 0, min(0, ymin)), (0, 0, 1.1 * max(abs(ymin), abs(ymax))), radius=0.005, color='black') # y-axis
            if showVolume == True:
                p += parametric_plot3d([abs(u - axisR) * cos(v), x1(u), axisR + abs(u - axisR) * sin(v)], (u, a, b), (v, 0, 2*pi), color='#ABB2B9', opacity=0.8) # surface 1
                p += parametric_plot3d([abs(u - axisR) * cos(v), x2(u), axisR + abs(u - axisR) * sin(v)], (u, a, b), (v, 0, 2*pi), color='#ABB2B9', opacity=0.8) # surface 2
                if x1(b) != x2(b):
                    p += parametric_plot3d([abs(b - axisR) * cos(v), min(x1(b), x2(b)) + t * (max(x1(b), x2(b)) - min(x1(b), x2(b))), axisR + abs(b - axisR) * sin(v)], (t, 0, 1), (v, 0, 2*pi), color='#ABB2B9', opacity=0.8) # outside
                if x1(a) != x2(a):
                    p += parametric_plot3d([abs(a - axisR) * cos(v), min(x1(a), x2(a)) + t * (max(x1(a), x2(a)) - min(x1(a), x2(a))), axisR + abs(a - axisR) * sin(v)], (t, 0, 1), (v, 0, 2*pi), color='#ABB2B9', opacity=0.8) # inside
            if showShells == True:
                if ystar == 'Lower endpoint':
                    yy = [a + dy * i for i in range(n)]   
                elif ystar == 'Upper endpoint':
                    yy = [a + dy * (i + 1) for i in range(n)] 
                elif ystar == 'Midpoint':
                    yy = [a + dy * (i + 1/2) for i in range(n)]
                else:
                    yy = [a + dy * (i + random()) for i in range(n)]
                p += sum(draw_washer( abs(a - axisR) + i * dy, abs(a - axisR) + (i + 1) * dy, 0, abs(x1(yy[i]) - x2(yy[i])), cc[mod(i, 5)], 0.8).rotate((1, 0, 0), pi/2).translate(0, min(x1(yy[i]), x2(yy[i])), axisR) for i in range(n))
                if n <= 5:
                    p += sum(line3d([(0, x1(yy[i]), yy[i]),(0, x2(yy[i]), yy[i])], radius=0.005, color='black') for i in range(n))
            show(p + curves, frame = False, viewer = 'threejs', aspect_ratio = 1)

<h2><font face="Comic Sans MS">Volume by Revolution: Cylindrical Shells</font></h2>

SW50ZXJhY3RpdmUgZnVuY3Rpb24gPGZ1bmN0aW9uIF8gYXQgMHgxZDMxOGY1ZjA+IHdpdGggMSB3aWRnZXQKICByb3RhdGlvbjogVG9nZ2xlQnV0dG9ucyhkZXNjcmlwdGlvbj11JyRBeGlzXFzigKY=


In [8]:
#SageWidget-Calculus-VolumeByRevolution-08-27-2019-1130-LU
from sage.plot.plot3d.shapes import *; var('x', 'y', 'z', 'u', 'v','w', 't', 's'); 
pretty_print(html('<h2><font face="Comic Sans MS">Volume by Revolution: Cylindrical Shells</font></h2>'))
def draw_washer(r, R, ax, h, c, o):
        w = point3d((0,0,0),size=5,color='white') # Show the origin
        if r < R:
            w += parametric_plot3d([u*sin(v), ax + u*cos(v), 0], (u, r, R), (v, 0, 2 * pi), color = c, opacity = o) # BOTTOM
            w += parametric_plot3d([u*sin(v), ax + u*cos(v), h], (u, r, R), (v, 0, 2 * pi), color = c, opacity = o) # TOP
            w += parametric_plot3d([R * sin(v), ax + R * cos(v), h * t], (t, 0, 1), (v, 0, 2 * pi), color = c, opacity = o) # OUTSIDE
            w += parametric_plot3d([r * sin(v), ax + r * cos(v), h * t], (t, 0, 1), (v, 0, 2 * pi), color = c, opacity = o) # INSIDE
        return w
@interact(layout = [['rotation']]) 
def _(rotation = selector(['|','--'], label=r'$Axis\ of\ Rotation$', buttons=True)):
    if rotation == '|':
        @interact(layout = [['axisR'], ['y1', 'y2', 'xint', 'space'], ['showVolume','showShells', 'n', 'xstar']])
        def _(axisR = input_box(0, width=3, label=r'$Axis\ of\ rotation: x=$'),
              y1 = input_box(x, width=20, label=r'$y_1=$'), 
              y2 = input_box(sqrt(x), width=20, label=r'$y_2=$'),
              xint = input_box((0, 1), width=10, label=r'$x:$'),
              xstar = selector(['Arbitrary', 'Lower endpoint', 'Upper endpoint', 'Midpoint'], label=r'$x^*$'),
              n = input_box(4, width=3, label=r'$n$'),
              showVolume = (r'$Volume$', True),
              showShells = (r'$Shells$', False)):
            (a, b) = xint
            dx = (b - a) / n
            y1 = y1 + 0 * x # avoid error when the integrand is a constant.
            y2 = y2 + 0 * x # avoid error when the integrand is a constant.
            y1 = y1.function(x)
            y2 = y2.function(x)
            p = line3d([(0, axisR, a - (b - a) / 2), (0, axisR, b + (b - a) / 2)], radius=0.001, color='black') # rotation axis
            p += arrow3d((0, 0, 0), (0, 0, 1.1 * y1(b)), radius=0.005, color='black') # x-axis
            p += arrow3d((0, 0, 0), (0, 1.2 * b, 0), radius=0.005, color='black') # y-axis
            p += parametric_plot3d([0, x, y1(x)], (x, a, b), radius=0.01, color='red')
            p += parametric_plot3d([0, x, y2(x)], (x, a, b), radius=0.01, color='blue')
            if showVolume == True:
                p += parametric_plot3d([abs(u - axisR) * cos(v), axisR + abs(u - axisR) * sin(v), y1(u)], (u, a, b), (v, 0, 2*pi), color='#ABB2B9', opacity=0.8) # surface 1
                p += parametric_plot3d([abs(u - axisR) * cos(v), axisR + abs(u - axisR) * sin(v), y2(u)], (u, a, b), (v, 0, 2*pi), color='#ABB2B9', opacity=0.8) # surface 2
                if y1(b) != y2(b):
                    p += parametric_plot3d([abs(b - axisR) * cos(v), axisR + abs(b - axisR) * sin(v), min(y1(b), y2(b)) + t * (max(y1(b), y2(b)) - min(y1(b), y2(b)))], (t, 0, 1), (v, 0, 2*pi), color='#ABB2B9', opacity=0.8) # outside
                if y1(a) != y2(a):
                    p += parametric_plot3d([abs(a - axisR) * cos(v), axisR + abs(a - axisR) * sin(v), min(y1(a), y2(a)) + t * (max(y1(a), y2(a)) - min(y1(a), y2(a)))], (t, 0, 1), (v, 0, 2*pi), color='#ABB2B9', opacity=0.8) # inside
            if showShells == True:
                if xstar == 'Lower endpoint':
                    xx = [a + dx * i for i in range(n)]   
                elif xstar == 'Upper endpoint':
                    xx = [a + dx * (i + 1) for i in range(n)] 
                elif xstar == 'Midpoint':
                    xx = [a + dx * (i + 1/2) for i in range(n)]
                else:
                    xx = [a + dx * (i + random()) for i in range(n)]
                p += sum(draw_washer( a + i * dx, a + (i + 1) * dx, axisR, abs(y1(xx[i]) - y2(xx[i])), 'orange', 0.8).translate(0, 0, min(y1(xx[i]), y2(xx[i]))) for i in range(n))
              #  if n <= 5:
               #     p += sum(line3d([(0, x1(yy[i]), yy[i]),(0, x2(yy[i]), yy[i])], radius=0.05, color='black') for i in range(n))
            show(p, frame = False, viewer = 'threejs', aspect_ratio = 1)
    elif rotation == '--':
        @interact(layout = [['axisR'], ['x1', 'x2', 'yint', 'space'], ['showVolume','showShells', 'n', 'ystar']])
        def _(axisR = input_box(0, width=3, label=r'$Axis\ of\ rotation: x=$'),
              x1 = input_box(y, width=20, label=r'$x_1=$'), 
              x2 = input_box(y^2, width=20, label=r'$x_2=$'),
              yint = input_box((0, 2), width=10, label=r'$y:$'),
              ystar = selector(['Arbitrary', 'Lower endpoint', 'Upper endpoint', 'Midpoint'], label=r'$y^*$'),
              n = input_box(4, width=3, label=r'$n$'),
              showVolume = (r'$Volume$', True),
              showShells = (r'$Shells$', False)):
            (a, b) = yint
            dy = (b - a) / n
            x1 = x1 + 0 * y # avoid error when the integrand is a constant.
            x2 = x2 + 0 * y # avoid error when the integrand is a constant.
            x1 = x1.function(y)
            x2 = x2.function(y)
            p = line3d([(0, a - (b - a) / 2, axisR), (0, b + (b - a) / 2, axisR)], radius=0.001, color='black') # rotation axis
            p += arrow3d((0, 0, 0), (0, 1.1 * x1(b), 0), radius=0.005, color='black') # x-axis
            p += arrow3d((0, 0, 0), (0, 0, 1.2 * b), radius=0.005, color='black') # y-axis
            p += parametric_plot3d([0, x1(y), y], (y, a, b), radius=0.01, color='red')
            p += parametric_plot3d([0, x2(y), y], (y, a, b), radius=0.01, color='blue')
            if showVolume == True:
                p += parametric_plot3d([abs(u - axisR) * cos(v), x1(u), axisR + abs(u - axisR) * sin(v)], (u, a, b), (v, 0, 2*pi), color='#ABB2B9', opacity=0.8) # surface 1
                p += parametric_plot3d([abs(u - axisR) * cos(v), x2(u), axisR + abs(u - axisR) * sin(v)], (u, a, b), (v, 0, 2*pi), color='#ABB2B9', opacity=0.8) # surface 2
                if x1(b) != x2(b):
                    p += parametric_plot3d([abs(b - axisR) * cos(v), min(x1(b), x2(b)) + t * (max(x1(b), x2(b)) - min(x1(b), x2(b))), axisR + abs(b - axisR) * sin(v)], (t, 0, 1), (v, 0, 2*pi), color='#ABB2B9', opacity=0.8) # outside
                if x1(a) != x2(a):
                    p += parametric_plot3d([abs(a - axisR) * cos(v), min(x1(a), x2(a)) + t * (max(x1(a), x2(a)) - min(x1(a), x2(a))), axisR + abs(a - axisR) * sin(v)], (t, 0, 1), (v, 0, 2*pi), color='#ABB2B9', opacity=0.8) # inside
            if showShells == True:
                if ystar == 'Lower endpoint':
                    yy = [a + dy * i for i in range(n)]   
                elif ystar == 'Upper endpoint':
                    yy = [a + dy * (i + 1) for i in range(n)] 
                elif ystar == 'Midpoint':
                    yy = [a + dy * (i + 1/2) for i in range(n)]
                else:
                    yy = [a + dy * (i + random()) for i in range(n)]
                p += sum(draw_washer( a + i * dy, a + (i + 1) * dy, axisR, abs(x1(yy[i]) - x2(yy[i])), 'orange', 0.8).rotate((1, 0, 0), pi/2).translate(0, min(x1(yy[i]), x2(yy[i])), 0) for i in range(n))
              #  if n <= 5:
               #     p += sum(line3d([(0, x1(yy[i]), yy[i]),(0, x2(yy[i]), yy[i])], radius=0.05, color='black') for i in range(n))
            show(p, frame = False, viewer = 'threejs', aspect_ratio = 1)

<h2><font face="Comic Sans MS">Volume by Revolution: Cylindrical Shells</font></h2>

SW50ZXJhY3RpdmUgZnVuY3Rpb24gPGZ1bmN0aW9uIF8gYXQgMHgxZTAxOTdkNzA+IHdpdGggMSB3aWRnZXQKICByb3RhdGlvbjogVG9nZ2xlQnV0dG9ucyhkZXNjcmlwdGlvbj11JyRBeGlzXFzigKY=
